In [3]:
## Problem Definition

# We are defining a system of medical vaccine delivery through use of drones
# We are using constraint programming for route selction and sequence of drop locations
# then we are using RRT to ....



# Project Start

## **Problem Background** 

The situation is we are using a vehicle (rover or drone) to deliver food kits from the basecamp to 5 destinations distributed around the base camp. Each destination has a weekly demand in terms of weight and volume. The vehicle also has certain weight and volume restrictions. We are assuming there are atleast 2 different type of vehicles which vary in terms of both weight and volume capacity. THe problem is to find a way to identify on a delivery plan to fulfil the demand at the lowest cost.

## **Solution Development and Topics References** 

We want to use 2 tools learned in class. The RRT/RRT* probabilistic planning and the constraint programming (MIP) methods for the same. The basic idea is to use the RRT/RRT* to determine the shortest distance from the base camp to each of the destinations and among all destinations. This distance will be later used as an input into the mathematical program.

Once the distance matrix is populated using RRT, we will go ahead and use mathematical programming to model the entire problem as a vechile routing program to optimize the number of vehicles to be used, the type of vehicle to be used, the sequence of destinations to be allocated and the total cost of operations. 

### **Tools to be used** 

RRT - Use assignment modules with some modification <br>
Linear Programming - Use PuLP package in python

## **Solution Start**

### **RRT**

**Input** : The map of the delivery region and the coordinates of the basecamp and the five destinations <br>
**Output** : The distance matrix between basecamp and destinations <br>
**Method** : Run RRT/RRT* among all points (basecamp and destinations) and populate with the shortest distance <br>

### **Mathematical Programming**

**Objective function** : <br>
The objective function is to minimize the total cost of operations. <br> 
Out intention is to have 2 components to total cost. <br>
Fixed cost : Fixed cost of using the vehicle.<br>
Variable Cost : Cost incurred by using the vehicle. This cost varies with the usage ( such as fuel costs ).<br>

**Parameters or Input to the Program** : <br>
1. Distance matrix from RRT <br> 
2. Fixed cost of using the truck USD/ trip <br>
3. Variable cost of using the truck ~ USD/ KM <br>
4. Averge speed of the vehicle ~ km/hr <br>
5. Weekly demand from each destination in terms of tons (tons) and volume (cubic meter)<br>
6. Capacity of truck in terms of tons and volume<br>
7. Eficiency of vehicle could be used ( % of weight or volume capacity could be used)<br>

**Decisions Variable** : <br>
1. X refers to the % of load of weekly demand moved from basecamp to differnet destinations ~ lies between 0 and 1 <br> 
2. Y refers to whether a truck is used or not ~ binary <br>


**Constraints** : <br>
1. No movement from destination i to detination i
2. Relationship between Y and X . If X > 0 for a trip in a day , then Y should be 1 for that trip, 0 otherwise
3. Sum across all X should be more than 100%. Weekly demand is met. 
4. Load in a truck within the capacity
5. Subtour elimination constraints ( to ensure that vehicle does not get into cycles and get stuck)
6. Balacning constraint ( at every destination what comes in goes out )


###### Below this line is only trials #

I can type **bold** and *italicized* text.

I can type equations:

$$\sum_{i=0}^{k}x_i$$

I can type in-line equations $x+y=z$.

In [ ]:
# Defining Inputs to the model ( Constants or Parameters ) 

In [17]:


n_drop_points = 10
n_drones = 5 # we need to define the drones in capacity with different
n_property = 2 # weight carrying capacity for medicines and number of vaccines in terms of nos ( cold storage )
n_trips = 5 # number of trips in a week possible 

# all sets definition
# set suppliers:= 1..n_suppliers;
# set trips:=1..n_trips;
# set truck:=1..n_trucks;
# set property := 1..n_property;
# set ARCS := {i in suppliers, j in suppliers : i == j} ; 

param distance {suppliers,suppliers}; #need to put matrix of distances . 
distance = [[0,419,310,25,49,26,59,26,25,25],
            [419,0,592,445,483,413,421,458,456,455],
            [310,592,0,335,358,288,278,335,333,333]
            [310,592,0,335,358,288,278,335,333,333]
            [25,445,335,0,30,51,113,7,3.9,3]
            [49,483,358,30,0,75,136,30,28,27]
            [26,413,288,51,75,0,80,52,49,49]
            [59,421,278,113,136,80,0,112,146,145]
            [26,458,335,7,30,52,112,0,4.7,5]
            [25,456,333,3.9,28,49,146,4.7,0,1.6]
            [25,455,333,3,27,49,145,5,1.6,0]
            ]
    
#param FCT {truck};
FCT = [542,577,577,577,577] # this is the fixed cost of running the drone

#param VC {truck};
VC = [32,28,28,28,28] # this is the variable cost of running the drone per km

#param S;
S = 35 #speed of the drone

#param demand {suppliers,property};
demand = [[0,0]
        ]

param teff;
drone_eff = 0.95 # hos much of the drone capacity we can use

param capacity {truck,property}; # how to put drone capacity here

tripCost = [100,100,100,100,100]

max_dist = 3200  #you dont want the drone to travel more than a ptricual distance than 16 hours a week. 

profit = 1.375 #Percentage margin

# Sample Pup Problem Below for reference

SyntaxError: invalid syntax (Temp/ipykernel_28604/2034830670.py, line 13)

In [1]:
from pulp import *

In [2]:
x1 = LpVariable("ChickenPercent", 0, None, LpInteger)
x2 = LpVariable("BeefPercent", 0)


In [6]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)

C:\Users\anups\.julia\conda\3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [7]:
# The objective function is added to 'prob' first
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredients per can"

In [8]:
# The five constraints are entered
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, "ProteinRequirement"
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, "FatRequirement"
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, "FibreRequirement"
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, "SaltRequirement"

In [9]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

[BeefPercent, ChickenPercent]

In [10]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [11]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [12]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

BeefPercent = 66.0
ChickenPercent = 34


In [13]:
# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  0.97
